In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 4 on 12 virtual cores
Environment:
  JULIA_NUM_THREADS = 4


## 9-2. スレッド

### 9-2-1. Julia でスレッドを利用する

#### コード9-21. `Threads.nthreads()` の実行結果（スレッドが有効でない場合）

```julia
julia> Threads.nthreads()
1
```

#### コード9-22. `-t` の使用例（数値を指定する場合）

```bash
$ julia -t 4  # `julia --threads 4` でも同じ
```

#### コード9-23. `-t` の使用例（数値を指定する場合）の動作確認

```julia
julia> Threads.nthreads()
4
```

#### コード9-24. `-t` の使用例（`auto` を指定する場合）

```bash
$ julia -t auto  # `julia --threads auto` でも同じ
```

#### コード9-25. `-t` の使用例（`auto` を指定する場合）の動作確認

```julia
julia> Threads.nthreads()
12  # 結果は環境によって異なります
```

#### コード9-26. Bash/Zsh など（Linux/macOS など）の環境変数指定方法

```bash
# グローバルに境変数を設定してから julia を起動する例
$ export JULIA_NUM_THREADS=4
$ julia

# 一時的な環境変数の指定と同時に julia を起動する例
$ JULIA_NUM_THREADS=4 julia
```

#### コード9-27. Powershell（Windows など）の環境変数指定方法

```shell
> $env:JULIA_NUM_THREADS=4
> julia
```

#### コード9-28. `-t` の使用例（環境変数 `JULIA_NUM_THREADS` と同時に指定した場合）の動作確認

```julia
julia> # Bash で `JULIA_NUM_THREADS=4 julia -t auto` 等として実行したと仮定

julia> ENV["JULIA_NUM_THREADS"]  # ←環境変数の指定はこれで確認できる
"4"

julia> Threads.nthreads()
12  # 環境によって変わります。
```

### 9-2-2. スレッドの基本

#### `Threads.@threads` マクロ

#### コード9-29. `@threads` マクロの利用例(1)

In [2]:
Threads.nthreads()

4

In [3]:
function fib(n)
    if n ≤ 1
        n
    else
        fib(n - 2) + fib(n - 1)
    end
end

fib (generic function with 1 method)

In [4]:
@time fib(40)

  0.402024 seconds


102334155

In [5]:
@time for _=1:4
    println(fib(40))
end

102334155
102334155
102334155
102334155
  1.618738 seconds (10.84 k allocations: 585.193 KiB, 0.87% compilation time)


In [6]:
@time Threads.@threads for _=1:4
    println(fib(40))
end

102334155
102334155
102334155
102334155
  0.451954 seconds (52.91 k allocations: 2.824 MiB, 6.90% compilation time)


#### コード9-30. `@threads` マクロの利用例(2)、および `Threads.threadid()` の例

In [7]:
Threads.nthreads()

4

In [8]:
Threads.@threads for i=1:10
    println("i: ", i, ", ThreadID: ", Threads.threadid(), ", ", fib(40))
end

i: 1, ThreadID: 1, 102334155
i: 7, ThreadID: 2, 102334155
i: 4, ThreadID: 3, 102334155
i: 9, ThreadID: 4, 102334155
i: 2, ThreadID: 1, 102334155
i: 8, ThreadID: 2, 102334155
i: 5, ThreadID: 3, 102334155
i: 10, ThreadID: 4, 102334155
i: 3, ThreadID: 1, 102334155
i: 6, ThreadID: 3, 102334155


#### `Threads.@spawn` マクロ

#### コード9-31. `Threads.@spawn` マクロの利用例(1)

In [9]:
Threads.nthreads()

4

In [10]:
@time @sync for _=1:4
    Threads.@spawn println(fib(40))
end

102334155
102334155
102334155
102334155
  0.437198 seconds (20.42 k allocations: 990.414 KiB, 4.70% compilation time)


In [11]:
@sync for x=1:2, y=3:4
    Threads.@spawn println((x, y))
end

(2, 3)
(1, 3)
(1, 4)
(2, 4)


#### `Threads.@threads` と `Threads.@spawn` の使い分け

#### コード9-32. `Threads.@threads` と `Threads.@spawn` の違い(1)

In [12]:
Threads.nthreads()

4

In [13]:
@time fib(15)

  0.000004 seconds


610

In [14]:
tidhist = zeros(Int, Threads.nthreads());

In [15]:
@time Threads.@threads for i=1:40000
    fib(15)  # 毎回ほぼ同じ負荷の処理が実行される例
    tidhist[Threads.threadid()] += 1
end

  0.063782 seconds (124.93 k allocations: 3.793 MiB, 58.63% compilation time)


In [16]:
tidhist

4-element Vector{Int64}:
 10000
 10000
 10000
 10000

In [17]:
tidhist = zeros(Int, Threads.nthreads());

In [18]:
@time @sync for i=1:40000
    Threads.@spawn begin
        fib(15)  # 毎回ほぼ同じ負荷の処理が実行される例
        tidhist[Threads.threadid()] += 1
    end
end

  0.056646 seconds (418.82 k allocations: 24.113 MiB, 13.34% gc time, 31.42% compilation time)


In [19]:
tidhist

4-element Vector{Int64}:
  4556
 12231
 11348
 11865

#### コード9-33. `Threads.@threads` と `Threads.@spawn` の違い(2)

In [20]:
Threads.nthreads()

4

In [21]:
@time fib(25)

  0.000299 seconds


75025

In [22]:
@time fib(35)

  0.036699 seconds


9227465

In [23]:
tidhist = zeros(Int, Threads.nthreads());

In [24]:
@time Threads.@threads for i=1:1000
    fib(rand(25:35))  # 毎回わずかに異なる負荷の処理が実行
    tidhist[Threads.threadid()] += 1
end

  2.307128 seconds (76.10 k allocations: 4.009 MiB, 1.99% compilation time)


In [25]:
tidhist

4-element Vector{Int64}:
 250
 250
 250
 250

In [26]:
tidhist = zeros(Int, Threads.nthreads());

In [27]:
@time @sync for i=1:1000
    Threads.@spawn begin
        fib(rand(25:35))  # 毎回わずかに異なる負荷の処理が実行
        tidhist[Threads.threadid()] += 1
    end
end

  2.303401 seconds (95.39 k allocations: 5.346 MiB, 0.71% compilation time)


In [28]:
tidhist

4-element Vector{Int64}:
 269
 243
 238
 250

#### `Channel(spawn=true)`

#### コード9-34. `Channel(spawn=true)` の例(1): 単純な動作確認例

In [29]:
Threads.nthreads()

4

In [30]:
fib(40)

102334155

In [31]:
chnl = Channel{Int}(spawn=true) do chnl
    for n = 36:40
        put!(chnl, fib(n))
    end
end;

In [32]:
for v in chnl
    println(v)
end

14930352
24157817
39088169
63245986
102334155


#### コード9-35. `Channel(spawn=true)` の例(2): `spawn=true` 未指定時との比較

In [33]:
@time begin
    chnl = Channel{Int}() do chnl
        for n = 36:40
            put!(chnl, fib(n))
        end
    end;
    for v in chnl
        @time println(fib(40) - v)
    end
end

87403803
  0.493617 seconds (37 allocations: 976 bytes)
78176338
  0.553717 seconds (137 allocations: 9.656 KiB)
63245986
  0.643076 seconds (137 allocations: 9.688 KiB)
39088169
  0.793784 seconds (137 allocations: 9.688 KiB)
0
  0.398534 seconds (139 allocations: 9.789 KiB)
  2.981214 seconds (27.00 k allocations: 1.369 MiB, 1.34% compilation time)


In [34]:
@time begin
    chnl = Channel{Int}(spawn=true) do chnl
        for n = 36:40
            put!(chnl, fib(n))
        end
    end;
    for v in chnl
        @time println(fib(40) - v)
    end
end

87403803
  0.405369 seconds (39 allocations: 1.016 KiB)
78176338
  0.404398 seconds (137 allocations: 9.719 KiB)
63245986
  0.407212 seconds (137 allocations: 9.719 KiB)
39088169
  0.422683 seconds (136 allocations: 9.688 KiB)
0
  0.402679 seconds (138 allocations: 9.773 KiB)
  2.132320 seconds (20.69 k allocations: 1.028 MiB, 1.00% compilation time)


#### `Threads.foreach()`

#### コード9-36. `Threads.foreach()` の例(1): 基本動作

In [35]:
Threads.nthreads()

4

In [36]:
for x=1:2, y=3:4
    println((x, y))
end

(1, 3)
(1, 4)
(2, 3)
(2, 4)


In [37]:
# ↑と同じ挙動をするコード
# ↓`foreach(println, [(x, y) for x=1:2 for y=3:4])` と書いても同じ
foreach([(x, y) for x=1:2 for y=3:4]) do (x, y)
    println((x, y))
end

(1, 3)
(1, 4)
(2, 3)
(2, 4)


In [38]:
# `Threads.@threads` を利用したマルチスレッド対応版
Threads.@threads for (x, y) in [(x, y) for x=1:2 for y=3:4]
    println((x, y))
end

(1, 3)
(2, 3)
(2, 4)
(1, 4)


In [39]:
# `Threads.foreach()` を利用する場合、まず `Channel` を準備
chnl = Channel() do chnl
    for x=1:2, y=3:4
        put!(chnl, (x, y))
    end
end;

In [40]:
# ↓ `Threads.foreach(println, chnl)` と書いても同じ
Threads.foreach(chnl) do (x, y)
    println((x, y))
end

(1, 3)
(2, 4)
(2, 3)
(1, 4)


#### コード9-37. `Threads.foreach()` の例(2): `Base.foreach()` と仕様を合わせる例

In [41]:
threaded_foreach(f, c::Channel) = Threads.foreach(f, c)

threaded_foreach (generic function with 1 method)

In [42]:
function threaded_foreach(f, itr)
    channel = Channel() do chnl
        for v in itr
            put!(chnl, v)
        end
    end
    Threads.foreach(f, channel)
end

threaded_foreach (generic function with 2 methods)

In [43]:
function threaded_foreach(f, itrs...)
    channel = Channel() do chnl
        for args in zip(itrs...)
            put!(chnl, args)
        end
    end
    Threads.foreach(args -> f(args...), channel)
end

threaded_foreach (generic function with 3 methods)

In [44]:
threaded_foreach(f) = foreach(f)

threaded_foreach (generic function with 4 methods)

In [45]:
foreach(println, 1:3)

1
2
3


In [46]:
threaded_foreach(println, 1:3)  # 実行する度に結果は変わります

1
3
2


In [47]:
foreach((x, y) -> println(x + y), 1:3, 4:6)

5
7
9


In [48]:
threaded_foreach((x, y) -> println(x + y), 1:3, 4:6)  # 実行する度に結果は変わります

7
5
9


#### コード9-38. `Threads.foreach()` の例(3): `schedule` キーワード引数の指定による挙動の差異(1)

In [49]:
Threads.nthreads()

4

In [50]:
@time fib(15)

  0.000004 seconds


610

In [51]:
tidhist = zeros(Int, Threads.nthreads());

In [52]:
@time begin
    chnl = Channel{Int}() do chnl
        for i=1:40000
            put!(chnl, i)
        end
    end
    Threads.foreach(chnl) do i
        fib(15)  # 毎回ほぼ同じ負荷の処理が実行される例
        tidhist[Threads.threadid()] += 1
    end
end

  0.136260 seconds (521.61 k allocations: 27.474 MiB, 39.77% compilation time)


In [53]:
tidhist

4-element Vector{Int64}:
  6299
 11361
 11003
 11337

In [54]:
tidhist = zeros(Int, Threads.nthreads());

In [55]:
@time begin
    chnl = Channel{Int}() do chnl
        for i=1:40000
            put!(chnl, i)
        end
    end
    Threads.foreach(chnl, schedule=Threads.StaticSchedule()) do i
        fib(15)  # 毎回ほぼ同じ負荷の処理が実行される例
        tidhist[Threads.threadid()] += 1
    end
end

  0.118810 seconds (237.21 k allocations: 7.182 MiB, 42.53% compilation time)


In [56]:
tidhist

4-element Vector{Int64}:
  6591
 11275
 10913
 11221

#### コード9-39. `Threads.foreach()` の例(3): `schedule` キーワード引数の指定による挙動の差異(2)

In [57]:
Threads.nthreads()

4

In [58]:
@time fib(15)

  0.000004 seconds


610

In [59]:
@time fib(25)

  0.000299 seconds


75025

In [60]:
tidhist = zeros(Int, Threads.nthreads());

In [61]:
@time begin
    chnl = Channel{Int}() do chnl
        for i=1:10000
            put!(chnl, i)
        end
    end
    Threads.foreach(chnl) do i
        fib(rand(15:25))  # 毎回わずかに異なる負荷の処理が実行される例
        tidhist[Threads.threadid()] += 1
    end
end

  0.386899 seconds (172.29 k allocations: 9.126 MiB, 10.54% compilation time)


In [62]:
tidhist

4-element Vector{Int64}:
 4176
 1971
 1920
 1933

In [63]:
tidhist = zeros(Int, Threads.nthreads());

In [64]:
@time begin
    chnl = Channel{Int}() do chnl
        for i=1:10000
            put!(chnl, i)
        end
    end
    Threads.foreach(chnl, schedule=Threads.StaticSchedule()) do i
        fib(rand(15:25))  # 毎回わずかに異なる負荷の処理が実行される例
        tidhist[Threads.threadid()] += 1
    end
end

  0.384014 seconds (94.44 k allocations: 3.683 MiB, 8.70% compilation time)


In [65]:
tidhist

4-element Vector{Int64}:
 4063
 1960
 1994
 1983

### 9-2-3. 実用例

#### ソートアルゴリズムの並列化

#### コード9-40. バイトニックソートアルゴリズム（シングルスレッド版、コード5-49. 再掲）

```julia
module BitonicSorterST

# ref: https://en.wikipedia.org/wiki/Bitonic_sorter

export BitonicSort

using Base.Order: Ordering, lt

struct BitonicSortAlg <: Base.Sort.Algorithm end
const BitonicSort = BitonicSortAlg()

function Base.sort!(x::AbstractVector, lo::Integer, hi::Integer, ::BitonicSortAlg, o::Ordering)
    lo ≥ hi && return x

    fullsize::Int = hi - lo
    d = sizeof(Int) * 8 - leading_zeros(fullsize - 1)  # == ceil(Int, log(2, fullsize))

    for p = 1:d, q = 1:p
        _sort_kernel!(x, lo, hi, p, q, o)
    end
    return x
end

function _sort_kernel!(x::AbstractVector, lo, hi, p, q, o)
    # @assert p ≥ q
    halfsize_1 = Int(hi - lo) >> 0x01
    d = 1 << UInt(p - q)
    for s = 0:halfsize_1
        ioff = s & (d - 1)
        seg = lo + (s ⊻ ioff) << 0x01
        joff = q == 1 ? (2d - ioff - 1) : ioff + d
        i = seg + ioff
        j = seg + joff
        if !lt(o, x[i], x[j])
            x[i], x[j] = x[j], x[i]
        end
    end
end

end # module
```

In [66]:
include("../Chapter5/BitonicSorterST.jl")

Main.BitonicSorterST

#### コード9-41. バイトニックソートアルゴリズム（マルチスレッド版）

```julia
module BitonicSorterMT

export BitonicSortMT

using Base.Order: Ordering, lt

struct BitonicSortMTAlg <: Base.Sort.Algorithm end
const BitonicSortMT = BitonicSortMTAlg()

function Base.sort!(x::AbstractVector, lo::Integer, hi::Integer, ::BitonicSortMTAlg, o::Ordering)
    lo ≥ hi && return x

    fullsize::Int = hi - lo
    d = sizeof(Int) * 8 - leading_zeros(fullsize - 1)  # == ceil(Int, log(2, fullsize))

    for p = 1:d, q = 1:p
        _sort_kernel!(x, lo, hi, p, q, o)
    end
    return x
end

function _sort_kernel!(x::AbstractVector, lo, hi, p, q, o)
    # @assert p ≥ q
    halfsize_1 = (hi - lo) >> 0x01
    d = 1 << UInt(p - q)
    Threads.@threads for s = 0:halfsize_1
        ioff = s & (d - 1)
        seg = lo + (s ⊻ ioff) << 0x01
        joff = q == 1 ? (2d - ioff - 1) : ioff + d
        i = seg + ioff
        j = seg + joff
        if !lt(o, x[i], x[j])
            x[i], x[j] = x[j], x[i]
        end
    end
end

end # module
```

In [67]:
include("BitonicSorterMT.jl")

Main.BitonicSorterMT

#### コード9-42. バイトニックソートの動作確認・比較

In [68]:
Threads.nthreads()

4

In [69]:
using .BitonicSorterST, .BitonicSorterMT

In [70]:
x_org = [10, 30, 11, 20, 4, 330, 21, 110];

In [71]:
# デフォルトアルゴリズムでソート
sort(x_org)

8-element Vector{Int64}:
   4
  10
  11
  20
  21
  30
 110
 330

In [72]:
# BitonicSortでソート
sort(x_org; alg=BitonicSort)

8-element Vector{Int64}:
   4
  10
  11
  20
  21
  30
 110
 330

In [73]:
# BitonicSortMTでソート
sort(x_org; alg=BitonicSortMT)

8-element Vector{Int64}:
   4
  10
  11
  20
  21
  30
 110
 330

In [74]:
x_org_2_16 = rand(Float64, 2^16);

In [75]:
sort(x_org_2_16) == 
sort(x_org_2_16; alg=BitonicSort) ==
sort(x_org_2_16; alg=BitonicSortMT)

true

In [76]:
using BenchmarkTools

In [77]:
@btime sort($x_org_2_16);

  2.987 ms (2 allocations: 512.05 KiB)


In [78]:
@btime sort($x_org_2_16; alg=BitonicSort);

  10.000 ms (2 allocations: 512.05 KiB)


In [79]:
@btime sort($x_org_2_16; alg=BitonicSortMT);

  3.392 ms (3407 allocations: 811.83 KiB)


#### `Iterators.map` のマルチスレッド化

#### コード9-43. `threaded_map()`（マルチスレッド版 `map()`）の実装例

In [80]:
threaded_map(f) = map(f)

threaded_map (generic function with 1 method)

In [81]:
function threaded_map(f, itr)
    ntasks = Threads.nthreads()
    intermediate_channel = Channel{Task}(ntasks; spawn=true) do chnl
        for arg in itr
            put!(chnl, Threads.@spawn(f(arg)))
        end
    end
    (fetch(task) for task in intermediate_channel)
end

threaded_map (generic function with 2 methods)

In [82]:
function threaded_map(f, itrs...)
    ntasks = Threads.nthreads()
    intermediate_channel = Channel{Task}(ntasks; spawn=true) do chnl
        for args in zip(itrs...)
            put!(chnl, Threads.@spawn(f(args...)))
        end
    end
    (fetch(task) for task in intermediate_channel)
end

threaded_map (generic function with 3 methods)

#### コード9-44. `threaded_map()`（マルチスレッド版 `map()`）の動作確認例

In [83]:
Threads.nthreads()

4

In [84]:
map(+, 1:3, 4:6)

3-element Vector{Int64}:
 5
 7
 9

In [85]:
collect(threaded_map(+, 1:3, 4:6))  # 結果（要素順序）も同じであることにも注目

3-element Vector{Int64}:
 5
 7
 9

In [86]:
function fib(n)
    if n ≤ 1
        n
    else
        fib(n - 2) + fib(n - 1)
    end
end

fib (generic function with 1 method)

In [87]:
map(fib, 40:-1:21)

20-element Vector{Int64}:
 102334155
  63245986
  39088169
  24157817
  14930352
   9227465
   5702887
   3524578
   2178309
   1346269
    832040
    514229
    317811
    196418
    121393
     75025
     46368
     28657
     17711
     10946

In [88]:
collect(threaded_map(fib, 40:-1:21))

20-element Vector{Int64}:
 102334155
  63245986
  39088169
  24157817
  14930352
   9227465
   5702887
   3524578
   2178309
   1346269
    832040
    514229
    317811
    196418
    121393
     75025
     46368
     28657
     17711
     10946

In [89]:
@time map(fib, 40:-1:21);

  1.055855 seconds (1 allocation: 224 bytes)


In [90]:
@time collect(threaded_map(fib, 40:-1:21));

  0.430626 seconds (199 allocations: 12.656 KiB)


#### コード9-45. `threaded_map_unordered()`の例

In [91]:
# 簡単のためイテレータを1つだけ受け取るもののみ実装
function threaded_map_unordered(f, itr)
    input_channel = Channel(spawn=true) do chnl
        for args in itr
            put!(chnl, args)
        end
    end
    ntasks = Threads.nthreads()
    Channel(ntasks; spawn=true) do chnl
        Threads.foreach(input_channel; ntasks=ntasks) do arg
            result = f(arg)
            put!(chnl, result)
        end
    end
end

threaded_map_unordered (generic function with 1 method)

In [92]:
collect(threaded_map_unordered(fib, 40:-1:21))  # 結果は順不同

20-element Vector{Any}:
  24157817
  14930352
  39088169
   5702887
   9227465
   3524578
   2178309
   1346269
    514229
    832040
    317811
    196418
    121393
     46368
     75025
     28657
     17711
     10946
  63245986
 102334155

In [93]:
@time collect(threaded_map_unordered(fib, 40:-1:21));

  0.411094 seconds (419 allocations: 20.562 KiB)


### 9-2-4. スレッドセーフ

#### コード9-46. スレッドアンセーフの例(1)：マルチスレッドでのフィールド同時更新

In [94]:
Threads.nthreads()

4

In [95]:
mutable struct UnsafeCounter
    count::Int
    UnsafeCounter() = new(0)
end

In [96]:
counter = UnsafeCounter();

In [97]:
for n=1:1000
    counter.count += 1
end

In [98]:
counter.count
# ↓これは期待する結果

1000

In [99]:
counter = UnsafeCounter();

In [100]:
Threads.@threads for n=1:1000
    counter.count += 1
end

In [101]:
counter.count
# ↓期待する結果よりも少ない値になってしまう

374

#### `Threads.@atomic` マクロ（Julia v1.7 以降のみ）

#### コード9-47. スレッドセーフの例(1)：`Threads.@atomic` による不可分操作

In [102]:
@assert VERSION ≥ v"1.7"
# ※注意：ここでエラーが発生するなら以降のコードは（一部）動作しません。

In [103]:
Threads.nthreads()

4

In [104]:
mutable struct AtomicCounter
    Threads.@atomic count::Int
    AtomicCounter() = new(0)
end

In [105]:
counter = AtomicCounter();

In [106]:
Threads.@threads for n=1:1000
    Threads.@atomic counter.count += 1
end

In [107]:
counter.count

1000

In [108]:
counter = AtomicCounter();

In [109]:
for n=1:1000
    Threads.@atomic counter.count += 1  # マルチスレッドでなくても `Threads.@atomic` 必須
end

In [110]:
counter.count

1000

#### ロック機構による排他制御(1)

#### コード9-48. スレッドセーフの例(2)：`Threads.SpinLock` による排他制御

In [111]:
Threads.nthreads()

4

In [112]:
mutable struct UnsafeCounter
    count::Int
    UnsafeCounter() = new(0)
end

In [113]:
counter = UnsafeCounter();

In [114]:
for n=1:1000
    counter.count += 1
end

In [115]:
counter.count

1000

In [116]:
lck = Threads.SpinLock();

In [117]:
counter = UnsafeCounter();

In [118]:
Threads.@threads for n=1:1000
    lock(lck)
    try
        counter.count += 1
    finally
        unlock(lck)
    end
end

In [119]:
counter.count

1000

In [120]:
lck = Threads.SpinLock();

In [121]:
counter = UnsafeCounter();

In [122]:
Threads.@threads for n=1:1000
    lock(lck) do
        counter.count += 1
    end
end

In [123]:
counter.count

1000

In [124]:
lck = Threads.SpinLock();

In [125]:
counter = UnsafeCounter();

In [126]:
Threads.@threads for n=1:1000
    # ↓ Julia v1.7 以降は単に `@lock` でもOK
    Base.@lock lck counter.count += 1
end

In [127]:
counter.count

1000

#### コード9-49. `Threads.SpinLock` を利用したスレッドセーフなカウンタ

In [128]:
mutable struct SpinLockCounter
    count::Int
    lock::Threads.SpinLock
    SpinLockCounter() = new(0, Threads.SpinLock())
end

In [129]:
next!(counter::SpinLockCounter) = 
    Base.@lock counter.lock counter.count += 1

counter = SpinLockCounter();

In [130]:
Threads.@threads for n=1:1000
    next!(counter)
end

In [131]:
counter.count

1000

#### ロック機構による排他制御(2)

#### コード9-50. スレッドアンセーフの例(2)：再帰を伴う辞書の更新

In [132]:
function update!(d::AbstractDict, n, counter::SpinLockCounter)
    get!(d, n) do
        # println("recursive call $(n)->$(n-1) (threadid: $(Threads.threadid()))")
        update!(d, n-1, counter)
        newcount = next!(counter)
        # println("(n, count): ($n, $newcount) (threadid: $(Threads.threadid()))")
        newcount
    end
end

update! (generic function with 1 method)

In [133]:
using Random

In [134]:
ps = randperm(10)

10-element Vector{Int64}:
  2
  5
  9
 10
  1
  3
  6
  7
  4
  8

In [135]:
dict = Dict(0 => 0);

In [136]:
counter = SpinLockCounter();

In [137]:
for n in ps
    update!(dict, n, counter)
end

In [138]:
dict

Dict{Int64, Int64} with 11 entries:
  5  => 5
  8  => 8
  1  => 1
  0  => 0
  6  => 6
  9  => 9
  3  => 3
  7  => 7
  4  => 4
  2  => 2
  10 => 10

In [139]:
dict = Dict(0 => 0);

In [140]:
counter = SpinLockCounter();

In [141]:
Threads.@threads for n in ps
    update!(dict, n, counter)
end

In [142]:
dict

Dict{Int64, Int64} with 11 entries:
  5  => 5
  8  => 8
  1  => 1
  0  => 0
  6  => 6
  9  => 9
  3  => 3
  7  => 7
  4  => 4
  2  => 2
  10 => 10

#### 仮想コード9-1. コード9-50. のコメントアウトを外して実行

```julia
julia> Threads.@threads for n in ps
           update!(dict, n, counter)
       end
recursive call 5->4 (threadid: 1)
recursive call 9->8 (threadid: 2)
recursive call 4->3 (threadid: 1)
recursive call 2->1 (threadid: 4)
recursive call 3->2 (threadid: 1)
recursive call 1->0 (threadid: 4)
recursive call 2->1 (threadid: 1)
(n, count): (1, 1) (threadid: 4)
recursive call 8->7 (threadid: 3)
  :《中略》
(n, count): (7, 14) (threadid: 3)
(n, count): (8, 15) (threadid: 3)
(n, count): (7, 16) (threadid: 2)
(n, count): (8, 17) (threadid: 2)
(n, count): (9, 18) (threadid: 2)
recursive call 10->9 (threadid: 2)
(n, count): (10, 19) (threadid: 2)
```

#### コード9-51. スレッドセーフの例(3)：再帰を伴う辞書の更新（`ReentrantLock` 利用）

In [143]:
function update!(d::AbstractDict, n, counter::SpinLockCounter, lck)
    lock(lck) do
        get!(d, n) do
            # println("recursive call $(n)->$(n-1) (threadid: $(Threads.threadid()))")
            update!(d, n-1, counter)
            newcount = next!(counter)
            # println("(n, count): ($n, $newcount) (threadid: $(Threads.threadid()))")
            newcount
        end
    end
end

update! (generic function with 2 methods)

In [144]:
ps  # 先ほどランダムシャッフルした数列

10-element Vector{Int64}:
  2
  5
  9
 10
  1
  3
  6
  7
  4
  8

In [145]:
dict = Dict(0 => 0);

In [146]:
counter = SpinLockCounter();

In [147]:
lck = ReentrantLock()  # `Threads.SpinLock()` ではNG

ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 139681948471808, 139681948472688))

In [148]:
Threads.@threads for n in ps
    update!(dict, n, counter, lck)
end

In [149]:
dict

Dict{Int64, Int64} with 11 entries:
  5  => 5
  8  => 8
  1  => 1
  0  => 0
  6  => 6
  9  => 9
  3  => 3
  7  => 7
  4  => 4
  2  => 2
  10 => 10